# Cleaning the raw dataset

## Summary

This cleans the raw `FundswithPrices` dataset, but relaxes restrictions to allow for some missing data.  The missing data will be interpolated using a group mean, which should preserve commonalities within each group and still allow detection of anomalies (if any exist).

Cleaning steps:

- Remove funds that don't exist as of 1/31/21
- Remove funds that don't have at least 36 months of data
- Remove funds that have constant price across time
- Select `fundSeries` with longest price history within each `fundName`
- Consolidate `aafmCategory` into fewer groups, increasing within-group sample size

In [2]:
from typing import Dict, List

import pandas as pd

In [3]:
data = pd.read_csv('../data/FundDatawithMonthlyPrices_v2_raw.csv')
print(data.shape)

(4795, 81)


In [4]:
data['aafmCategory'].isna().sum()

11

In [5]:
data.dropna(subset=['aafmCategory'], inplace=True)

## Minimum History and Non-Constant Variance

Let's filter out observations that have **at least** 36 months of data and still exist in the most recent month.  We'll also filter out any observations whose price does not change across the entire time period.

In [6]:
min_history = 36
mask = (data.iloc[:, -min_history:].notnull().sum(1) == min_history) & (data.iloc[:, 8:].apply(lambda x: x.nunique(), axis=1) != 1)
data = data[mask]
data.shape

(1822, 81)

In [7]:
data.head()

,fundRUN,fundName,fundSeries,aafmCategory,svsCategory,svsCategoryId,currency,fundRUNSeries,1/31/15,2/28/15,...,4/30/20,5/31/20,6/30/20,7/31/20,8/31/20,9/30/20,10/31/20,11/30/20,12/31/20,1/31/21
1,8812-9,A. CHILE CALIFICADO,AC,Inversionistas Calificados Accionario Nacional,FM DIRIGIDO A INVERSIONISTAS CALIFICADOS,8.0,P,8812-9AC,NaN,NaN,...,830.3527,767.4051,820.6198,836.8227,794.7268,758.6292,730.2904,831.3485,863.5786,867.1403
2,8812-9,A. CHILE CALIFICADO,AC-APV,Inversionistas Calificados Accionario Nacional,FM DIRIGIDO A INVERSIONISTAS CALIFICADOS,8.0,P,8812-9AC-APV,NaN,NaN,...,1039.3819,1039.3819,1039.3819,1039.3819,1039.3819,1039.3819,1039.3819,1039.3819,1039.3819,1039.3819
5,8812-9,A. CHILE CALIFICADO,D,Inversionistas Calificados Accionario Nacional,FM DIRIGIDO A INVERSIONISTAS CALIFICADOS,8.0,P,8812-9D,767.0540,788.5753,...,706.7567,665.9628,712.7872,727.2119,690.8105,659.9857,636.7984,728.8420,764.2218,769.6162
6,8812-9,A. CHILE CALIFICADO,E,Inversionistas Calificados Accionario Nacional,FM DIRIGIDO A INVERSIONISTAS CALIFICADOS,8.0,P,8812-9E,746.0934,766.5560,...,552.8584,510.9303,546.2861,557.0110,528.8156,504.7108,485.7939,552.8705,574.1806,576.4414
7,8812-9,A. CHILE CALIFICADO,F,Inversionistas Calificados Accionario Nacional,FM DIRIGIDO A INVERSIONISTAS CALIFICADOS,8.0,P,8812-9F,726.3573,746.5076,...,550.4048,508.8351,544.2244,555.0967,527.1768,503.3117,484.6114,551.7055,573.1648,575.6172


In [8]:
data['fundName'].nunique()

311

## Selecting a single series

### Choosing series by history length

These next two cells will select a single series, for each fund, based on the number of non-null observations. We end up with **311** unique observations.

In [9]:
def get_longest_series2(df: pd.DataFrame) -> pd.DataFrame:
    longest = []
    index_cols = list(df.columns[:8])
    df = df.copy().set_index(index_cols)
    for fund in df.index.get_level_values(1).unique():
        least_missing = df.xs(fund, level=1, drop_level=False).isna().sum(1).sort_values().index[0]
        longest.append(df.loc[least_missing, :])
    new = pd.concat(longest, axis=1).T
    new.index.names = index_cols
    return new.reset_index()

In [10]:
longest_history = get_longest_series2(data)
longest_history

,fundRUN,fundName,fundSeries,aafmCategory,svsCategory,svsCategoryId,currency,fundRUNSeries,1/31/15,2/28/15,...,4/30/20,5/31/20,6/30/20,7/31/20,8/31/20,9/30/20,10/31/20,11/30/20,12/31/20,1/31/21
0,8812-9,A. CHILE CALIFICADO,D,Inversionistas Calificados Accionario Nacional,FM DIRIGIDO A INVERSIONISTAS CALIFICADOS,8.0,P,8812-9D,767.0540,788.5753,...,706.7567,665.9628,712.7872,727.2119,690.8105,659.9857,636.7984,728.8420,764.2218,769.6162
1,8289-9,ACCIONES CHILE,APV,Accionario Nacional Large CAP,FM DE INVERSION EN INSTRUMENTOS DE CAPITALIZACION,5.0,P,8289-9APV,857.0135,882.9958,...,686.6956,629.7418,675.7632,688.7364,652.1686,629.9546,607.5724,684.7373,710.2978,725.5830
2,8076-4,ACCIONES CHILENAS,APV,Accionario Nacional Large CAP,FM DE INVERSION EN INSTRUMENTOS DE CAPITALIZACION,5.0,P,8076-4APV,4264.7434,4467.0361,...,4026.5817,3646.4398,3938.0199,3999.2216,3752.6015,3618.8839,3515.6612,3980.6780,4105.8619,4205.8988
3,9254-1,ACCIONES EUROPA,ALTO,Accionario Europa Desarrollado,FM DE INVERSION EN INSTRUMENTOS DE CAPITALIZACION,5.0,P,9254-1ALTO,NaN,NaN,...,911.2628,932.6800,977.8669,926.3185,992.1376,960.9146,872.7992,1041.4013,1006.0475,1025.7878
4,8247-3,ACCIONES LATAM,A,Accionario America Latina,FM DE INVERSION EN INSTRUMENTOS DE CAPITALIZACION,5.0,P,8247-3A,1693.6139,1722.7516,...,1397.8727,1480.2771,1610.8085,1663.5947,1604.6280,1499.6090,1470.9928,1785.2915,1797.7134,1767.1439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,8304-6,VALOR PLUS,A,Fondos de Deuda < 365 Dias Nacional en pesos,FM DE INV.EN INST.DE DEUDA DE C/P CON DURACION...,2.0,P,8304-6A,1634.0381,1640.8835,...,1918.2670,1928.0308,1926.6139,1925.9346,1934.1314,1931.6995,1937.9650,1944.9532,1943.1137,1949.1619
307,8991-5,VALOR PLUS UF,PERSO,Fondos de Deuda < 365 Dias Nacional en UF,FM DE INV.EN INST.DE DEUDA DE C/P CON DURACION...,2.0,P,8991-5PERSO,NaN,NaN,...,1178.1592,1186.1080,1182.1184,1180.8745,1190.5819,1190.0365,1197.4432,1205.2516,1202.1768,1206.7065
308,8011-K,VISION MONEY MARKET,100,Fondos de Deuda < 90 Dias Nacional,FM DE INV.EN INST.DE DEUDA DE C/P CON DURACION...,1.0,P,8011-K100,28476.9327,28546.6508,...,33027.1002,33027.9904,33034.4678,33076.0493,33079.9903,33106.6821,33120.5551,33173.1571,33265.5057,33260.7546
309,8346-1,ZURICH PATRIMONIO,A,"Fondos de Deuda > 365 Dias Nacional, Inversion...",FM DE INV.EN INST.DE DEUDA DE MEDIANO Y LARGO ...,3.0,P,8346-1A,1376.8601,1390.0569,...,1559.5532,1559.9891,1563.8820,1564.2163,1594.6716,1565.7276,1588.0307,1607.2982,1615.9960,1642.3596


### Choosing series by total assets

These cells will select a single series, for each fund, based on the `netPatrimony` or total fund value. We end up with **269** unique observations.

**I'm not sure why** this approach results in fewer unique observations...perhaps it's in the merge.

In [11]:
aum = pd.read_parquet('../data/fund_flows.parq')
aum.set_index('fecha', inplace=True)
aum.index = pd.to_datetime(aum.index)
series_max = aum.loc['2021-01-31', ['fundName', 'netPatrimony']].reset_index().groupby(['fecha', 'fundName']).idxmax().values.ravel()
max_aum = aum.loc['2021-01-31'].reset_index().loc[series_max, ['fundName', 'fundSeries']]
max_aum.shape

(441, 2)

In [12]:
most_assets = data.merge(max_aum, on=['fundName', 'fundSeries'])
most_assets

,fundRUN,fundName,fundSeries,aafmCategory,svsCategory,svsCategoryId,currency,fundRUNSeries,1/31/15,2/28/15,...,4/30/20,5/31/20,6/30/20,7/31/20,8/31/20,9/30/20,10/31/20,11/30/20,12/31/20,1/31/21
0,8812-9,A. CHILE CALIFICADO,E,Inversionistas Calificados Accionario Nacional,FM DIRIGIDO A INVERSIONISTAS CALIFICADOS,8.0,P,8812-9E,746.0934,766.5560,...,552.8584,510.9303,546.2861,557.0110,528.8156,504.7108,485.7939,552.8705,574.1806,576.4414
1,8289-9,ACCIONES CHILE,PERSO,Accionario Nacional Large CAP,FM DE INVERSION EN INSTRUMENTOS DE CAPITALIZACION,5.0,P,8289-9PERSO,1338.9739,1376.1419,...,905.4164,828.1450,886.4099,901.0582,850.9795,819.9076,788.7026,886.6157,917.3003,934.5757
2,8076-4,ACCIONES CHILENAS,EJECU,Accionario Nacional Large CAP,FM DE INVERSION EN INSTRUMENTOS DE CAPITALIZACION,5.0,P,8076-4EJECU,3796.5548,3971.7308,...,3290.6321,2975.8975,3209.6087,3255.0357,3050.1334,2937.5570,2849.8681,3222.5529,3319.3531,3395.5805
3,9254-1,ACCIONES EUROPA,UNIVE,Accionario Europa Desarrollado,FM DE INVERSION EN INSTRUMENTOS DE CAPITALIZACION,5.0,P,9254-1UNIVE,NaN,NaN,...,783.4517,799.8244,836.5092,790.5623,844.9480,816.6854,740.2300,881.4190,849.6989,864.5337
4,9537-0,ACCIONES MID CAP,EJECU,Accionario Nacional Otros,FM DE INVERSION EN INSTRUMENTOS DE CAPITALIZACION,5.0,P,9537-0EJECU,NaN,NaN,...,588.0875,577.5388,626.3344,626.6982,640.1492,594.8131,544.7290,596.5639,608.1711,630.0623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,8515-4,USA,CLASI,Accionario EEUU,FM MIXTO,4.0,D,8515-4CLASI,109.2709,114.4880,...,146.5487,152.0653,154.4945,162.9913,174.5510,167.8437,162.4720,179.5220,185.4046,184.6127
264,8113-2,USA EQUITY,C,Accionario EEUU,FM DE INVERSION EN INSTRUMENTOS DE CAPITALIZACION,5.0,P,8113-2C,1736.3360,1784.3087,...,3237.3682,3250.7098,3380.5086,3300.1248,3636.7854,3559.2764,3392.5425,3734.0498,3595.8769,3707.5678
265,8304-6,VALOR PLUS,A,Fondos de Deuda < 365 Dias Nacional en pesos,FM DE INV.EN INST.DE DEUDA DE C/P CON DURACION...,2.0,P,8304-6A,1634.0381,1640.8835,...,1918.2670,1928.0308,1926.6139,1925.9346,1934.1314,1931.6995,1937.9650,1944.9532,1943.1137,1949.1619
266,8991-5,VALOR PLUS UF,PERSO,Fondos de Deuda < 365 Dias Nacional en UF,FM DE INV.EN INST.DE DEUDA DE C/P CON DURACION...,2.0,P,8991-5PERSO,NaN,NaN,...,1178.1592,1186.1080,1182.1184,1180.8745,1190.5819,1190.0365,1197.4432,1205.2516,1202.1768,1206.7065


## Choosing a category

### Funds by AAFM Category

Here's a count of funds within each `aafmCategory`.  We don't have enough within-group observations to run anomaly detection algorithms for this grouping.

In [13]:
longest_history.groupby('fundName').first().groupby('aafmCategory').count().iloc[:, 0]

aafmCategory
Accionario America Latina                                             11
Accionario Asia Emergente                                              8
Accionario Asia Pacifico                                               1
Accionario Brasil                                                      4
Accionario Desarrollado                                               12
Accionario EEUU                                                       12
Accionario Emergente                                                   8
Accionario Europa Desarrollado                                         8
Accionario Europa Emergente                                            1
Accionario Nacional Large CAP                                         23
Accionario Nacional Otros                                              5
Accionario Pais                                                        7
Accionario Países MILA                                                 3
Accionario Sectorial                  

### Funds by SVS Category

Here's a count of funds within each `svsCategory`.  We *might* have enough within-group observations for about half of these groupings:

In [14]:
longest_history.groupby('fundName').first().groupby('svsCategory').count().iloc[:, 0]

svsCategory
FM DE INV.EN INST.DE DEUDA DE C/P CON DURACION <= 365 DIAS     19
FM DE INV.EN INST.DE DEUDA DE C/P CON DURACION <= 90 DIAS      42
FM DE INV.EN INST.DE DEUDA DE MEDIANO Y LARGO PLAZO            50
FM DE INVERSION EN INSTRUMENTOS DE CAPITALIZACION              70
FM DE LIBRE INVERSION                                         100
FM DIRIGIDO A INVERSIONISTAS CALIFICADOS                        5
FM MIXTO                                                       25
Name: fundRUN, dtype: int64

### Funds by Consolidated AAFM Category 

In [15]:
new_categories = {
    'International Equity': [
        'Accionario America Latina', 'Accionario Asia Pacifico', 'Accionario Brasil', 'Accionario Desarrollado', 'Accionario EEUU', 'Accionario Europa Desarrollado',
        'Accionario Pais', 'Accionario Países MILA',
    ],
    'Emerging Equity': [
        'Accionario Asia Emergente', 'Accionario Emergente', 'Accionario Europa Emergente',
    ],
    'Domestic Equity': [
        'Accionario Nacional Large CAP', 'Accionario Nacional Otros', 'Accionario Sectorial',
    ],
    'Balanced, Conservative': ['Balanceado Conservador'],
    'Balanced, Moderate': ['Balanceado Moderado'],
    'Balanced, Aggressive': ['Balanceado Agresivo'],
    'International Bond, < 365': [
        'Fondos de Deuda < 365 Dias Internacional', 'Fondos de Deuda < 90 Dias Internacional, Dolar',  
    ],
    'Domestic Bond, < 365': [
        'Fondos de Deuda < 365 Dias Nacional en UF', 'Fondos de Deuda < 365 Dias Nacional en pesos', 'Fondos de Deuda < 90 Dias Nacional',
        'Inversionistas Calificados Accionario Nacional',
    ],
    'International Bond, > 365': [
        'Fondos de Deuda > 365 Dias Internacional, Mercados Emergentes', 'Fondos de Deuda > 365 Dias Internacional, Mercados Internacionales',
        
    ],
    'Domestic Bond, > 365': [
        'Fondos de Deuda > 365 Dias Nacional, Inversion UF > 3 años y =<5', 'Fondos de Deuda > 365 Dias Nacional, Inversion en UF > 5 años',
        'Fondos de Deuda > 365 Dias Nacional, Inversión en Pesos', 'Fondos de Deuda > 365 Dias Nacional, Inversión en UF < 3 años',
        'Fondos de Deuda > 365 Dias Orig. Flex', 'Inversionistas Calificados Títulos de Deuda',
    ]
}

In [16]:
def lookup(val: str, cats: Dict[str, List[str]]) -> str:
    try:
        res = [x for x in cats.keys() if val in cats[x]]
        return res[0]
    except IndexError:
        print(f'{val} not in lookup.')

### Map existing AAFM categories to consolidated categories

In [17]:
new = longest_history['aafmCategory'].apply(lambda x: lookup(x, new_categories))
longest_history.insert(loc=8, column='userCategory', value=new)

In [18]:
longest_history.groupby('fundName').first().groupby('userCategory').count().iloc[:, 0]

userCategory
Balanced, Aggressive         22
Balanced, Conservative       21
Balanced, Moderate           22
Domestic Bond, < 365         55
Domestic Bond, > 365         58
Domestic Equity              30
Emerging Equity              17
International Bond, < 365    10
International Bond, > 365    18
International Equity         58
Name: fundRUN, dtype: int64

In [19]:
longest_history.to_csv('../data/FundDataWithMonthlyPrices_v3.csv', index=False)

### Get Average Fund Size for 2020

In [20]:
aum = pd.read_parquet('../data/fund_flows.parq')
aum.set_index('fecha', inplace=True)
aum.index = pd.to_datetime(aum.index)

In [36]:
assets = aum.loc['2020']
assets['fundRUNSeries'] = assets['fundRUN'] + assets['fundSeries']
assets = assets.groupby('fundRUNSeries').mean()
assets.to_csv('../data/FundDataRecentFlows.csv')

<ipython-input-36-f5f5b7563078>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assets['fundRUNSeries'] = assets['fundRUN'] + assets['fundSeries']
